## Generative AI PDF information exctractor
In this use case it is shown how to exctract information from a PDF file through LLM queries. For this use case is necessary the use of a vector database (in this case FAISS), embeddings and OpenAI model calls.

In [2]:
from dotenv import load_dotenv
import os

load_dotenv("apis.env")

True

In [4]:
pdf_path = "TM6_digital_manual_MES-es-ES_prefill_20190121.pdf"

# Load pdf with external info not seen during training of the LLM
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

In [5]:
pages[0]

Document(page_content='MANUAL DE\nINSTRUCCIONES\nTHERMOMIX® TM6\nV 2.0 • DIGITAL MANUAL', metadata={'source': 'TM6_digital_manual_MES-es-ES_prefill_20190121.pdf', 'page': 0})

In [6]:
# Generate vector space representation with words from the external data
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [7]:
# Load embeddings in vector database
from langchain.vectorstores import FAISS
db = FAISS.from_documents(pages, embeddings)

In [8]:
# Use information retrieval from embedding for answer
from langchain.chains import RetrievalQA
from langchain import OpenAI

llm = OpenAI()
chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever())

In [9]:
# Query 1
query = "Cómo puedo pesar ingredientes?"
chain(query, return_only_outputs=True)

{'result': ' Para pesar ingredientes, coloque el vaso en la unidad central y toque el icono de la báscula del menú Funciones y herramientas. Pese el primer ingrediente (hasta máx. 3,0 kg). Si desea añadir más ingredientes, pulse el icono de tara y añada el siguiente ingrediente al vaso. Repita estos pasos tantas veces como sea necesario hasta alcanzar el peso máximo (6 kg). La variación hasta 3 kg es de +/–30 g.'}

In [10]:
# Query 2
query = "Puedo hacer helado con la thermomix?"
chain(query, return_only_outputs=True)

{'result': ' No, la thermomix no tiene la función para hacer helado.'}

In [11]:
# Query 3
query = "Existe algún modo rápido para triturar alimentos?"
chain(query, return_only_outputs=True)

{'result': ' Sí, puedes utilizar las velocidades de 4 a 10 para trocear porciones grandes, medianas y pequeñas y para mezclar y triturar. También puedes utilizar el modo "Turbo" para poner la unidad central a velocidad máxima.'}

In [12]:
# Query 4
query = "Necesito hacer ajo en polvo, cómo lo hago?"
chain(query, return_only_outputs=True)

{'result': ' No sabemos cómo hacer ajo en polvo.'}